## Instalasi Library

In [ ]:
!pip install scrapy
!pip install pandas
!pip install Sastrawi

## Menghapus File lama

In [7]:
!rm -rf ptautm_scrape.py hasil.json && sleep 1

## Menulis Script Konfigurasi Scrapy

In [8]:
%%writefile -a ptautm_scrape.py
import string
import scrapy
from scrapy import Request

class ptaUTM(scrapy.Spider):
    name = "Abstract"
    start_urls = [
      'https://pta.trunojoyo.ac.id/c_search/byprod/10'
      ]
    def parse(self, response):
        for url in response.xpath('//li[contains(@data-id,"id-1")]/div[3]/a/@href'):
            yield response.follow(url, callback=self.parse_content)

        next_page = response.xpath('(//li/a[contains(@class, "pag_button")])[last() -1]/@href').get()
        if next_page != 'https://pta.trunojoyo.ac.id/c_search/byprod/10/20':
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

    def parse_content(self, response):
        
        item = {
            'judul' : response.xpath('//a[contains(@class, "title")]/text()').get(),
            'abstrak' : response.xpath('(//p[contains(@align, "justify")])[1]/text()').get()
        }

        yield item

Writing ptautm_scrape.py


## Menjalankan Script Scrapy

In [9]:
!scrapy runspider ptautm_scrape.py -O hasil.json && sleep 2

2022-09-03 04:08:27 [scrapy.utils.log] INFO: Scrapy 2.6.2 started (bot: scrapybot)
2022-09-03 04:08:27 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.1.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.4.0, Python 3.7.13 (default, Apr 24 2022, 01:04:09) - [GCC 7.5.0], pyOpenSSL 22.0.0 (OpenSSL 3.0.5 5 Jul 2022), cryptography 37.0.4, Platform Linux-5.4.188+-x86_64-with-Ubuntu-18.04-bionic
2022-09-03 04:08:27 [scrapy.crawler] INFO: Overridden settings:
{'SPIDER_LOADER_WARN_ONLY': True}
2022-09-03 04:08:27 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2022-09-03 04:08:27 [scrapy.extensions.telnet] INFO: Telnet Password: 1a29e42326eaba46
2022-09-03 04:08:27 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2022-09-03 04:08:27

## Import Hasil

In [61]:
import pandas as pd
df = pd.read_json('hasil.json')
df['abstrak'][3]

'Penjadwalan  kuliah  di  Perguruan  Tinggi  merupakan  masalah yang  kompleks.  Dalam  permasalahan tersebut  terdapat  beberapa variabel  yang  harus  dipertimbangkan  supaya  mendapatkan  jadwal  yang \r\noptimal. Di dalam penyusunan jadwal kuliah ini terdapat sangat banyak kemungkinan  yang  selayaknya  dicoba  untuk  menemukan  penjadwalan yang  terbaik.  Karena  itu  dibutuhkan  metode  optimasi  yang  dapat diterapkan  untuk  mengerjakan  penjadwalan  mata  kuliah  ini.  Metode yang  dapat  digunakan  untuk  menyelesaikan  permasalahan  tersebut adalah  dengan  menggunakan  Algoritma  Genetika.  Tugas  Akhir  ini \r\nbertujuan  untuk  membuat  suatu  sistem  yang  dapat  membuat  jadwal matakuliah  secara  otomatis  dan  hasilnya  optimal.  Dari  hasil  penelitian yang  dilakukan  algoritma  genetika  mampu  membuat  jadwal  dengan \r\noptimal  yang  dibuktikan  dari  nilai  fitness  yang memiliki  nilai  1  secara otomatis  jadwal  yang  dibentuk  akan  optimal.  Semoga  adanya

## Case Folding

In [62]:
import string

#mengubah menjadi huruf kecil
df['abstrak'] = df['abstrak'].str.lower()

#menghapus tanda baca
for char in string.punctuation:
    df['abstrak'] = df['abstrak'].replace(r'[\%s]'%char," ", regex=True)

#menghapus angka
df['abstrak'] = df['abstrak'].replace(r'\d+',' ', regex=True)
#menghapus karakter kosong
df['abstrak'] = df['abstrak'].replace(r'\s+',' ', regex=True)
df['abstrak'][3]



'penjadwalan kuliah di perguruan tinggi merupakan masalah yang kompleks dalam permasalahan tersebut terdapat beberapa variabel yang harus dipertimbangkan supaya mendapatkan jadwal yang optimal di dalam penyusunan jadwal kuliah ini terdapat sangat banyak kemungkinan yang selayaknya dicoba untuk menemukan penjadwalan yang terbaik karena itu dibutuhkan metode optimasi yang dapat diterapkan untuk mengerjakan penjadwalan mata kuliah ini metode yang dapat digunakan untuk menyelesaikan permasalahan tersebut adalah dengan menggunakan algoritma genetika tugas akhir ini bertujuan untuk membuat suatu sistem yang dapat membuat jadwal matakuliah secara otomatis dan hasilnya optimal dari hasil penelitian yang dilakukan algoritma genetika mampu membuat jadwal dengan optimal yang dibuktikan dari nilai fitness yang memiliki nilai secara otomatis jadwal yang dibentuk akan optimal semoga adanya aplikasi ini diharapkan bisa mempermudah didalam pembuatan penjadwalan matakuliah dengan mudah dan cepat kata k

## Stopword Removal

In [63]:
import urllib.request, json 
with urllib.request.urlopen("https://raw.githubusercontent.com/smilesense/stopwords-id/master/stopwords-id.json") as list_stopwords:
    data_stopword = json.load(list_stopwords)

for i in data_stopword:
    df['abstrak'] = df['abstrak'].replace(r'\b%s\b'%i, '', regex=True)
df['abstrak'] = df['abstrak'].replace(r'\s+',' ', regex=True)

df['abstrak'][3]

'penjadwalan kuliah perguruan kompleks permasalahan variabel dipertimbangkan jadwal optimal penyusunan jadwal kuliah selayaknya dicoba menemukan penjadwalan terbaik dibutuhkan metode optimasi diterapkan penjadwalan mata kuliah metode menyelesaikan permasalahan algoritma genetika tugas bertujuan sistem jadwal matakuliah otomatis hasilnya optimal hasil penelitian algoritma genetika jadwal optimal dibuktikan nilai fitness memiliki nilai otomatis jadwal dibentuk optimal semoga aplikasi diharapkan mempermudah didalam pembuatan penjadwalan matakuliah mudah cepat kunci algoritma genetika optimasi penjadwalan '

## Stemming

In [64]:
# import StemmerFactory class
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()
# stemming process

ind = 0
for sentence in df['abstrak']:
    df['abstrak'][ind] = stemmer.stem(str(sentence))
    ind+=1
df['abstrak'][3]

'jadwal kuliah guru kompleks masalah variabel timbang jadwal optimal susun jadwal kuliah layak coba temu jadwal baik butuh metode optimasi terap jadwal mata kuliah metode selesai masalah algoritma genetika tugas tuju sistem jadwal matakuliah otomatis hasil optimal hasil teliti algoritma genetika jadwal optimal bukti nilai fitness milik nilai otomatis jadwal bentuk optimal moga aplikasi harap mudah dalam buat jadwal matakuliah mudah cepat kunci algoritma genetika optimasi jadwal'

## Tokenisasi

In [65]:
listkata = []
for abstrak in df['abstrak']:
    listkata = listkata + str(abstrak).split()

listkata_nodup = list(dict.fromkeys(listkata))
hasil_hitung = {}
for test1 in listkata_nodup:
    jumlah = 0
    for test2 in range(len(listkata)):
        if test1 == listkata[test2]:
            jumlah+=1
    hasil_hitung.update({'%s'%test1 : jumlah})
    
hasil_hitung = dict(sorted(hasil_hitung.items(), key=lambda item: item[1], reverse=True))
# print(hasil_hitung)

## Term

In [69]:
# for k in hasil_hitung.copy():
#     if hasil_hitung[k] < 10:
#         del hasil_hitung[k]

def terms(dataframe):
  df2 = dataframe.copy()
  print("Masukkan Jumlah Terms yang ingin ditambahkan (max %s)"%len(hasil_hitung))
  max_term = int(input())
  a = 1
  for inter in range(len(df2['abstrak'])):
      for fitur in hasil_hitung:
          df2['term-%s (%s)'%(a,fitur)] = 0

          if a >= max_term:
            a = 1
            break
          a+=1

  for inter in range(len(df2['abstrak'])):
    for fitur in hasil_hitung:
        cek = (df2['abstrak'][inter]).split()
        jumlah = 0
        for iter2 in range(len(cek)):
          if fitur == cek[iter2]:
            jumlah+=1
        df2['term-%s (%s)'%(a,fitur)][inter] = jumlah
        if a >= max_term:
          a = 1
          break
        a+=1

  with pd.option_context('display.max_colwidth', None,
                        'display.max_columns', None,
                        'display.max_rows', None,
                        "mode.sim_interactive", True):
      display(df2)

terms(df)



Masukkan Jumlah Terms yang ingin ditambahkan (max 1857)
11


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,judul,abstrak,term-1 (hasil),term-2 (sistem),term-3 (metode),term-4 (proses),term-5 (data),term-6 (citra),term-7 (nilai),term-8 (aplikasi),term-9 (teliti),term-10 (uji),term-11 (butuh)
0,PERANCANGAN DAN IMPLEMENTASI SISTEM DATABASE \r\nTERDISTRIBUSI MENGGUNAKAN ORACLE STUDI KASUS \r\nSIAKAD UNIVERSITAS TRUNOJOYO,sistem informasi akademik siakad sistem informasi fungsi tangan kelola saji data data akademik fakultas siakad anggap layan mahasiswa butuh informasi akademik universitas trunojoyo sedia siakad database pusat sistem lebih awat mudah butuh biaya sistem potensi mengahadapi kendala kendala proses transaksi data padat jaring database siakad lambat pemrosesan respon query data simpan pemrosesan kompleks milik lemah sedia data sistem kembang sistem database sistem databases distribusi fakultas jadi solusi masalah basisdata distribusi untung milik basisdata pusat awas distribusi reability availability cepat pemrosesan query otonomi local,0,7,0,1,5,0,0,0,0,0,2
1,APLIKASI KONTROL DAN MONITORING JARINGAN KOMPUTER BERBASIS MOBILE,jalan koneksi jaring komputer lancar ganggu harap guna tanggung bidang administrator jaring koneksi jaring jalan lancar ganggu nyata lapang jaring komputer skala potensi ganggu rusa koneksi jaring komputer sebab banyak guna padat traffic data jaring komputer jaga tingkat kualitas layan koneksi jaring akses kontrol monitoring level administrator jaring real time bas mobile atur informasi koneksi jaring akses administrator jaring tuntut ganggu putus koneksi jaring administrator kontrol monitoring sistem kontrol monitoring bas mobile harap bantu tugas administrator kontrol pantau jaring ambil bijak node node koneksi jaring manajemen koneksi port bandwidth mobile benah ganggu jaring pakai jaring klien adu ganggu kunci administrator jaring koneksi jaring akses kontrol monitoring,0,1,0,0,1,0,0,0,0,0,0
2,RANCANG BANGUN APLIKASI PROXY SERVER UNTUK\r\nENKRIPSI KODE HTML MENGGUNAKAN\r\nALGORITMA BLOWFISH,web server perangkat lunak server fungsi terima minta http https web client kirim bentuk halaman halaman web bentuk dokumen html dokumen html kirim dokumen bentuk bahasa markup plaintext dokumen isi kode kode html murni proses enkripsi tuju penelitan aplikasi proxy server terima minta http web client terus web server aplikasi terima respon server proses enkripsi dokumen html dokumen html kirim web client bentuk dokumen html enkripsi web client aplikasi internet explorer mozilla firefox opera aplikasi aplikasi kenal sebut web browser web browser terima dokumen html enkripsi aplikasi proxy server javascript engine web browser proses dekripsi rendering dokumen tampil sesuai asli kode dokumen html menu view source web browser dapat dokumen html enkripsi,0,0,0,3,0,0,0,6,0,0,0
3,SISTEM PENDUKUNG KEPUTUSAN OPTIMASI PENJADWALAN MATAKULIAH UNTUK PENGGUNAAN RUANGAN KULIAH BERSAMA DI UNIVERSITAS TRUNOJOYO MADURA MENGGUNAKAN ALGORITMA \r\nGENETIKA,jadwal kuliah guru kompleks masalah variabel timbang jadwal optimal susun jadwal kuliah layak coba temu jadwal baik butuh metode optimasi terap jadwal mata kuliah metode selesai masalah algoritma genetika tugas tuju sistem jadwal matakuliah otomatis hasil optimal hasil teliti algoritma genetika jadwal optimal bukti nilai fitness milik nilai otomatis jadwal bentuk optimal moga aplikasi harap mudah dalam buat jadwal matakuliah mudah cepat kunci algoritma genetika optimasi jadwal,2,1,2,0,0,0,2,1,1,0,1
4,IMPLEMENTASI ALGORITMA PRIM DAN DEPTH FIRST SEARCH PADA PEMBUATAN MAZE GAME BERBASIS ANDROID OS MOBILE,teknologi mobile game beroperating system open source kembang pesat ragam variasi mobile game milik minat kalang dasar kembang kembang mobile game teliti mobile game asah mampu berfikir kembang implementasi algoritma prim depth first search buat algoritma prim terap buat lintas maze algoritma depth first search terap proses cari solusi hasil uji simpul algoritma prim depth first search fungsi resolusi screen x pixels bentuk lintas cari solusi maze kunci mobile game android algoritma p

In [67]:
df2.to_csv(r'my_data.csv', index=False)

There is a lot more that you can do with outputs (such as including interactive outputs)
with your book. For more information about this, see [the Jupyter Book documentation](https://jupyterbook.org)